In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import pingouin as pg
import plotly.graph_objects as go
from os.path import join as pjoin

sys.path.append("../..")
import circletrack_behavior as ctb
import plotting_functions as pf
import place_cells as pc

def set_groups(df): 
    df['group'] = df['mouse']
    df['group'] = df['group'].replace({'mca24': 'Young Adult', 'mca25': 'Young Adult', 'mca26': 'Young Adult', 'mca27': 'Young Adult', 
                                       'mca28': 'Middle Age', 'mca29': 'Middle Age', 'mca30': 'Middle Age', 'mca31': 'Middle Age', 
                                       'mca32': 'Aged', 'mca33': 'Aged', 'mca34': 'Aged', 'mca35': 'Aged', 
                                       'mca36': 'Young Adult', 'mca37': 'Young Adult', 'mca38': 'Young Adult', 'mca39': 'Young Adult', 
                                       'mca40': 'Middle Age', 'mca41': 'Middle Age', 'mca42': 'Middle Age', 'mca43': 'Middle Age',
                                       'mca44': 'Aged', 'mca45': 'Aged', 'mca46': 'Aged', 'mca47': 'Aged'})
    return df

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


### The code below can be used to visualize results after each day.

In [ ]:
## Set path variables
behavior_type = 'circletrack_data'
csv_name = 'circle_track.csv'
project_dir = 'MultiCon_Behavior'
experiment_dir = 'MultiCon_Aging2'
dpath = ''
dpath = f'../../../{project_dir}/{experiment_dir}/{behavior_type}/data/**/**/**/{csv_name}'
fig_path = (f'../../../{project_dir}/{experiment_dir}/intermediate_figures/')
group_colors = ['red', 'midnightblue', 'darkgrey']
group_colors_dict = {'Young Adult': 'darkgrey', 'Middle Age': 'midnightblue', 'Aged': 'red',
                     'Young Adult Thresh3': 'turquoise', 'Young Adult Thresh5': 'midnightblue'}
chance_color = 'darkgrey'

if not os.path.exists(fig_path):
    os.makedirs(fig_path)

## Create file list dataframe
file_list = ctb.get_file_list(dpath)
mouse_id = []
for file in file_list:
    mouse_id.append(ctb.get_mouse(file, str2match='(mca[0-9]+)'))
combined = ctb.combine(file_list, mouse_id)

In [ ]:
lick_df = pd.DataFrame()
for mouse in np.unique(mouse_id):
    lick_data = ctb.get_lick_accuracy(combined, mouse)
    lick_df = pd.concat([lick_df, lick_data], ignore_index=True)
lick_df = set_groups(lick_df)
## Lick accuracy plot
fig = pf.plot_behavior_across_days(lick_df, x_var='day', y_var='percent_correct', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=group_colors, avg_color='darkgrey', plot_datapoints=True,
                                   x_title='Day', y_title='Lick Accuracy (%)', titles=['Circle Track: Lick Accuracy'], height=700, width=700)
fig.show()
# fig.write_image(pjoin(fig_path, 'lick_accuracy.png'))
# lick_df.mixed_anova(dv='percent_correct', subject='mouse', within='day', between='group')

In [ ]:
reward_df = pd.DataFrame()
for mouse in np.unique(mouse_id):
    reward_data = ctb.get_total_rewards(combined, mouse)
    reward_df = pd.concat([reward_df, reward_data], ignore_index=True)
reward_df = set_groups(reward_df)
## Total rewards plot
fig = pf.plot_behavior_across_days(reward_df, x_var='day', y_var='total_rewards', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=group_colors, avg_color='darkgrey', expert_line=False, chance=False, plot_datapoints=True,
                                   x_title='Day', y_title='Total Rewards', titles=['Circle Track: Rewards'], height=700, width=700)
fig.show()
# fig.write_image(pjoin(fig_path, 'total_rewards.png'))

### Set up run order for the day.

In [ ]:
day = 'Day 38' ## edit depending on day
retired_mice = [] ## add retired mouse to this list
maze_info = pd.read_csv(f'../../../{project_dir}/{experiment_dir}/maze_yml/maze_info.csv')
todays_mazes = pd.read_csv(f'../../../{project_dir}/{experiment_dir}/maze_yml/{experiment_dir} - TodaysMazes.csv') ## download at end of the day
run_dict = {'maze1': [], 'maze2': [], 'maze3': [], 'maze4': []}
letter_dict = {'maze1': [], 'maze2': [], 'maze3': [], 'maze4': []}
for mouse in np.unique(todays_mazes['Mouse']):
    if mouse in retired_mice:
        pass
    else:
        maze_letter = todays_mazes.loc[todays_mazes['Mouse'] == mouse, day].reset_index(drop=True)

        if 'A' in maze_letter[0]:
            maze_number = maze_info.loc[maze_info['Mouse'] == mouse, "if 'A'"].reset_index(drop=True)
        elif 'B' in maze_letter[0]:
            maze_number = maze_info.loc[maze_info['Mouse'] == mouse, "if 'B'"].reset_index(drop=True)
        elif 'C' in maze_letter[0]:
            maze_number = maze_info.loc[maze_info['Mouse'] == mouse, "if 'C'"].reset_index(drop=True)
        elif 'D' in maze_letter[0]:
            maze_number = maze_info.loc[maze_info['Mouse'] == mouse, "if 'D'"].reset_index(drop=True)

        if maze_number[0] == 1:
            run_dict['maze1'].append(mouse)
            letter_dict['maze1'].append(maze_letter[0])
        elif maze_number[0] == 2:
            run_dict['maze2'].append(mouse)
            letter_dict['maze2'].append(maze_letter[0])
        elif maze_number[0] == 3:
            run_dict['maze3'].append(mouse)
            letter_dict['maze3'].append(maze_letter[0])
        elif maze_number[0] == 4:
            run_dict['maze4'].append(mouse)
            letter_dict['maze4'].append(maze_letter[0])
## Convert to dataframe to paste into excel         
run_df = pd.DataFrame.from_dict(run_dict, orient='index').T 
letter_df = pd.DataFrame.from_dict(letter_dict, orient='index').T

In [ ]:
run_df

In [ ]:
letter_df

### Analyze preprocessed behavior data.

In [2]:
## Settings
project_dir = 'MultiCon_Behavior'
experiment_dir = 'MultiCon_Aging2'
lin_path = f'../../../{project_dir}/{experiment_dir}/output/lin_behav/'
circle_path = f'../../../{project_dir}/{experiment_dir}/output/behav/'
fig_path = f'../../../{project_dir}/{experiment_dir}/intermediate_figures'
young_adult = [f'mca{x}' for x in np.arange(24, 28)] + [f'mca{x}' for x in np.arange(36, 40)]
middle_aged = [f'mca{x}' for x in np.arange(28, 32)] + [f'mca{x}' for x in np.arange(40, 44)]
aged = [f'mca{x}' for x in np.arange(32, 36)] + [f'mca{x}' for x in np.arange(44, 48)]
three_color_plots = ['red', 'midnightblue', 'darkgrey']
group_colors_dict = {'Young Adult': 'darkgrey', 'Middle Aged': 'midnightblue', 'Aged': 'red',
                     'Young Adult Thresh1': 'darkgrey', 'Young Adult Thresh5': 'black',
                     'Middle Aged Thresh1': 'turquoise', 'Middle Aged Thresh5': 'midnightblue',
                     'Aged Thresh1': 'red', 'Aged Thresh5': 'darkred'}
chance_color = 'darkgrey'
bin_size = 4 ## for binning across trials
error_colors = ['rgba(169,169,169,0.4)', 'rgba(169,169,169,0.4)', 'rgba(169,169,169,0.4)'] ## rgba is the only way to get the error band transparent
opacity = 0.8 ## for binning across trials

### Linear track results.

In [ ]:
## Linear track rewards
result_dict = {'mouse': [], 'day': [], 'group': [], 'rewards': []}
for mouse in os.listdir(lin_path):
    mouse_path = pjoin(lin_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        result_dict['mouse'].append(mouse)
        result_dict['day'].append(idx+1)
        result_dict['group'].append(group)
        result_dict['rewards'].append(np.sum(behav['water']))
results_df = pd.DataFrame(result_dict)

In [ ]:
## Plot rewards across days in linear track
fig = pf.plot_behavior_across_days(results_df, x_var='day', y_var='rewards', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=three_color_plots, avg_color='darkgrey', expert_line=False, chance=False,
                                   x_title='Day', y_title='Rewards', titles=['Linear Track'], height=500, width=500)
fig.show()
results_df.mixed_anova(dv='rewards', within='day', subject='mouse', between='group')
fig.write_image(pjoin(fig_path, 'linear_track_rewards.png'))
results_df.mixed_anova(dv='rewards', subject='mouse', within='day', between='group')

### Analyze circle track lick accuracy.

In [ ]:
## Looking at first licks
circletrack_results = {'mouse': [], 'day': [], 'group': [], 'session': [], 'lick_accuracy_thresh1': [], 'lick_accuracy_thresh5': [], 'rewards': []}
for mouse in os.listdir(circle_path)[:-1]:
    mouse_path = pjoin(circle_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    for idx, session in enumerate(os.listdir(mouse_path)):
            behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
            behav_data = behav[~behav['probe']]
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            pc_thresh1 = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=1, by_trials=False)
            pc_thresh5 = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=5, by_trials=False)
            circletrack_results['mouse'].append(mouse)
            circletrack_results['day'].append(idx+1)
            circletrack_results['group'].append(group)
            circletrack_results['session'].append(np.unique(behav['session'])[0])
            circletrack_results['lick_accuracy_thresh1'].append(pc_thresh1)
            circletrack_results['lick_accuracy_thresh5'].append(pc_thresh5)
            circletrack_results['rewards'].append(np.sum(behav_data['water']))
ct_df = pd.DataFrame(circletrack_results)

In [ ]:
## Plot lick accuracy across days for thresh of 1
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='lick_accuracy_thresh1', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=three_color_plots, avg_color='darkgrey', expert_line=True, chance=True,
                                   x_title='Day', y_title='Lick Accuracy', titles=['Circle Track'], height=500, width=500)
fig.update_yaxes(range=[0, 100], dtick=25)
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy_thresh1.png'))
ct_df.mixed_anova(dv='lick_accuracy_thresh1', within='day', subject='mouse', between='group')

In [ ]:
## Plot lick accuracy across days for thresh of 5
fig = pf.plot_behavior_across_days(ct_df, x_var='day', y_var='lick_accuracy_thresh5', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=three_color_plots, avg_color='darkgrey', expert_line=True, chance=True,
                                   x_title='Day', y_title='Lick Accuracy', titles=['Circle Track'], height=500, width=500)
fig.update_yaxes(range=[0, 100], dtick=25)
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy_thresh5.png'))
ct_df.mixed_anova(dv='lick_accuracy_thresh5', within='day', subject='mouse', between='group')

In [ ]:
## Look at lick accuracy just when a mouse is in A.
y_var = 'lick_accuracy_thresh5'
a_data = ct_df[ct_df['session'] == 'A']
fig = pf.plot_behavior_across_days(a_data, x_var='day', y_var=y_var, groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=three_color_plots, avg_color='darkgrey', expert_line=False, chance=True, plot_datapoints=False,
                                   x_title='Day', y_title='Lick Accuracy', titles=['Circle Track'], height=500, width=500)
fig.update_yaxes(range=[0, 100], dtick=25)
fig.show()
fig.write_image(pjoin(fig_path, f'{y_var}_in_A.png'))
a_data.mixed_anova(dv=y_var, within='day', subject='mouse', between='group')

In [ ]:
## Look at lick accuracy just when a mouse is in A.
y_var = 'rewards'
a_data = ct_df[ct_df['session'] == 'A']
fig = pf.plot_behavior_across_days(a_data, x_var='day', y_var=y_var, groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=three_color_plots, avg_color='darkgrey', expert_line=False, chance=True, plot_datapoints=False,
                                   x_title='Day', y_title='Rewards', titles=['Circle Track'], height=500, width=500)
fig.update_yaxes(dtick=25)
fig.show()
fig.write_image(pjoin(fig_path, f'{y_var}_in_A.png'))
a_data.mixed_anova(dv=y_var, within='day', subject='mouse', between='group')

### Look at trial by trial differences in lick accuracy for the first or fifth lick.

In [ ]:
## Calculate for lick accuracy across trials
data = {'mouse': [], 'day': [], 'context_day': [], 'group': [], 'trial_acc_1': [], 'trial_acc_5': []}
for mouse in os.listdir(circle_path)[:-1]:
    mouse_path = pjoin(circle_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    day_counter = 1
    session_id = np.nan
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav_data = pd.read_feather(pjoin(mouse_path, f'{session}'))
        reward_one, reward_two = np.unique(behav_data['reward_one'])[0], np.unique(behav_data['reward_two'])[0]
        pc_1 = ctb.lick_accuracy(behav_data, reward_one, reward_two, lick_threshold=1, by_trials=True)
        pc_5 = ctb.lick_accuracy(behav_data, reward_one, reward_two, lick_threshold=5, by_trials=True)
        binned_pc_1 = ctb.bin_data(pc_1, bin_size=bin_size)
        binned_pc_5 = ctb.bin_data(pc_5, bin_size=bin_size)
        data['mouse'].append(mouse)
        data['day'].append(idx+1)
        data['group'].append(group)
        data['trial_acc_1'].append(binned_pc_1)
        data['trial_acc_5'].append(binned_pc_5)

        if np.unique(behav['session'])[0] != session_id:
            day_counter = 1
            session_id = np.unique(behav['session'])[0]
            circletrack_results['context_day'].append(day_counter)
        else:
            day_counter += 1
            session_id = np.unique(behav['session'])[0]
            circletrack_results['context_day'].append(day_counter)
trial_acc = pd.DataFrame(data)

In [ ]:
group_data = {'Young Adult': [], 'Middle Aged': [], 'Aged': [], 'day': []}
for day in np.unique(trial_acc['day']):
    group_data['Young Adult'].append(np.nanmean(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Young Adult') & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Middle Aged'].append(np.nanmean(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Middle Aged') & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Aged'].append(np.nanmean(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Aged') & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Young Adult'].append(np.nanstd(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Young Adult') & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1) / np.sqrt(len(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Young Adult') & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['Middle Aged'].append(np.nanstd(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Middle Aged') & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1) / np.sqrt(len(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Middle Aged') & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['Aged'].append(np.std(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Aged') & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1) / np.sqrt(len(trial_acc['trial_acc_1'][(trial_acc['group'] == 'Aged') & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['day'].append(day)
    group_data['day'].append(day)
group_df = pd.DataFrame(group_data)

In [ ]:
## For lick accuracy first lick
only_min_trials = False
fig = pf.custom_graph_template(x_title='', y_title='', height=800, width=1200,
                               rows=2, columns=6, shared_x=True, shared_y=True,
                               titles=[f'Day {x}' for x in np.arange(1, len(np.unique(group_df['day']))+1)])
for idx, day in enumerate(np.unique(group_df['day'])):
    plot_data = group_df[group_df['day'] == day]

    if only_min_trials:
        trial_lengths = []
        for group in plot_data.columns[:-1]:
            trial_lengths.append(len(plot_data.loc[:, group].values[0]))
        min_trials = np.min(trial_lengths)
    else:
        min_trials = -1
    
    for ind, group in enumerate(plot_data.columns[:-1]):
        group_data = plot_data.loc[:, group].values[0][:min_trials]
        upper = plot_data.loc[:, group].values[0][:min_trials] + plot_data.loc[:, group].values[1][:min_trials]
        lower = plot_data.loc[:, group].values[0][:min_trials] - plot_data.loc[:, group].values[1][:min_trials]
        x_data = np.arange(1, len(group_data)+1) * bin_size
        if idx < 6:
            row, col = 1, idx + 1
        elif (idx >= 6) & (idx < 12):
            row, col = 2, idx - 5
        else:
            row, col = 3, idx - 11
        fig.add_trace(go.Scatter(x=x_data, y=group_data, mode='lines', opacity=opacity,
                                line_color=group_colors_dict[group], showlegend=False,
                                legendgroup=group, name=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Upper Bound', x=x_data, y=upper, mode='lines',
                                 marker=dict(color=error_colors[ind]), line=dict(width=0), showlegend=False, 
                                 legendgroup=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Lower Bound', x=x_data, y=lower, marker=dict(color=error_colors[ind]),
                                 line=dict(width=0), mode='lines', fillcolor=error_colors[ind], fill='tonexty', 
                                 showlegend=False, legendgroup=group), row=row, col=col)
fig.add_hline(y=75, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy', range=[0, 100], dtick=25, col=1)
fig.update_xaxes(title='Trial', row=2)
fig.show()
if only_min_trials:
    fig.write_image(pjoin(fig_path, 'lick_acc_min_trials_across_groups.png'))
else:
    fig.write_image(pjoin(fig_path, 'lick_acc_across_groups.png'))

In [ ]:
group_data = {'Young Adult': [], 'Middle Aged': [], 'Aged': [], 'day': []}
for day in np.unique(trial_acc['day']):
    group_data['Young Adult'].append(np.nanmean(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Young Adult') & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Middle Aged'].append(np.nanmean(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Middle Aged') & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Aged'].append(np.nanmean(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Aged') & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Young Adult'].append(np.std(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Young Adult') & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Young Adult') & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['Middle Aged'].append(np.std(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Middle Aged') & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Middle Aged') & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['Aged'].append(np.std(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Aged') & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_5'][(trial_acc['group'] == 'Aged') & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['day'].append(day)
    group_data['day'].append(day)
group_df_five = pd.DataFrame(group_data)

In [ ]:
## For lick accuracy fifth lick
only_min_trials = False
fig = pf.custom_graph_template(x_title='', y_title='', height=800, width=1200,
                               rows=3, columns=6, shared_x=True, shared_y=True,
                               titles=[f'Day {x}' for x in np.arange(1, len(np.unique(group_df_five['day']))+1)])
for idx, day in enumerate(np.unique(group_df_five['day'])):
    plot_data = group_df_five[group_df_five['day'] == day]

    if only_min_trials:
        trial_lengths = []
        for group in plot_data.columns[:-1]:
            trial_lengths.append(len(plot_data.loc[:, group].values[0]))
        min_trials = np.min(trial_lengths)
    else:
        min_trials = -1
    
    for ind, group in enumerate(plot_data.columns[:-1]):
        group_data = plot_data.loc[:, group].values[0][:min_trials]
        upper = plot_data.loc[:, group].values[0][:min_trials] + plot_data.loc[:, group].values[1][:min_trials]
        lower = plot_data.loc[:, group].values[0][:min_trials] - plot_data.loc[:, group].values[1][:min_trials]
        x_data = np.arange(1, len(group_data)+1) * bin_size
        if idx < 6:
            row, col = 1, idx + 1
        elif (idx >= 6) & (idx < 12):
            row, col = 2, idx - 5
        else:
            row, col = 3, idx - 11
        fig.add_trace(go.Scatter(x=x_data, y=group_data, mode='lines', opacity=opacity,
                                line_color=group_colors_dict[group], showlegend=False,
                                legendgroup=group, name=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Upper Bound', x=x_data, y=upper, mode='lines',
                                 marker=dict(color=error_colors[ind]), line=dict(width=0), showlegend=False, 
                                 legendgroup=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Lower Bound', x=x_data, y=lower, marker=dict(color=error_colors[ind]),
                                 line=dict(width=0), mode='lines', fillcolor=error_colors[ind], fill='tonexty', 
                                 showlegend=False, legendgroup=group), row=row, col=col)
fig.add_hline(y=75, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy', range=[0, 100], dtick=25, col=1)
fig.update_xaxes(title='Trial', row=3)
fig.show()
# if only_min_trials:
#     fig.write_image(pjoin(fig_path, 'lick_acc_min_trials_across_groups.png'))
# else:
#     fig.write_image(pjoin(fig_path, 'lick_acc_across_groups.png'))

### Look at trial by trial changes in lick accuracy comparing threshold 1 vs 5 for each group.

In [ ]:
group_of_interest = 'Young Adult'
group_data = {'Young Adult Thresh1': [], 'Young Adult Thresh5': [], 'day': []}
for day in np.unique(trial_acc['day']):
    group_data['Young Adult Thresh1'].append(np.nanmean(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Young Adult Thresh5'].append(np.nanmean(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Young Adult Thresh1'].append(np.std(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['Young Adult Thresh5'].append(np.std(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['day'].append(day)
    group_data['day'].append(day)
young_df = pd.DataFrame(group_data)

In [ ]:
## For lick accuracy fifth lick
only_min_trials = False
fig = pf.custom_graph_template(x_title='', y_title='', height=800, width=1200,
                               rows=3, columns=6, shared_x=True, shared_y=True,
                               titles=[f'Day {x}' for x in np.arange(1, len(np.unique(young_df['day']))+1)])
for idx, day in enumerate(np.unique(young_df['day'])):
    plot_data = young_df[young_df['day'] == day]

    if only_min_trials:
        trial_lengths = []
        for group in plot_data.columns[:-1]:
            trial_lengths.append(len(plot_data.loc[:, group].values[0]))
        min_trials = np.min(trial_lengths)
    else:
        min_trials = -1
    
    for ind, group in enumerate(plot_data.columns[:-1]):
        group_data = plot_data.loc[:, group].values[0][:min_trials]
        upper = plot_data.loc[:, group].values[0][:min_trials] + plot_data.loc[:, group].values[1][:min_trials]
        lower = plot_data.loc[:, group].values[0][:min_trials] - plot_data.loc[:, group].values[1][:min_trials]
        x_data = np.arange(1, len(group_data)+1) * bin_size
        if idx < 6:
            row, col = 1, idx + 1
        elif (idx >= 6) & (idx < 12):
            row, col = 2, idx - 5
        else:
            row, col = 3, idx - 11
        fig.add_trace(go.Scatter(x=x_data, y=group_data, mode='lines', opacity=opacity,
                                line_color=group_colors_dict[group], showlegend=False,
                                legendgroup=group, name=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Upper Bound', x=x_data, y=upper, mode='lines',
                                 marker=dict(color=error_colors[ind]), line=dict(width=0), showlegend=False, 
                                 legendgroup=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Lower Bound', x=x_data, y=lower, marker=dict(color=error_colors[ind]),
                                 line=dict(width=0), mode='lines', fillcolor=error_colors[ind], fill='tonexty', 
                                 showlegend=False, legendgroup=group), row=row, col=col)
fig.add_hline(y=75, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy', range=[0, 100], dtick=25, col=1)
fig.update_xaxes(title='Trial', row=3)
fig.show()
if only_min_trials:
    fig.write_image(pjoin(fig_path, 'lick_acc_min_trials_young.png'))
else:
    fig.write_image(pjoin(fig_path, 'lick_acc_across_young.png'))

In [ ]:
group_of_interest = 'Middle Aged'
group_data = {'Middle Aged Thresh1': [], 'Middle Aged Thresh5': [], 'day': []}
for day in np.unique(trial_acc['day']):
    group_data['Middle Aged Thresh1'].append(np.nanmean(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Middle Aged Thresh5'].append(np.nanmean(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Middle Aged Thresh1'].append(np.std(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['Middle Aged Thresh5'].append(np.std(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['day'].append(day)
    group_data['day'].append(day)
middle_aged_df = pd.DataFrame(group_data)

In [ ]:
## For lick accuracy fifth lick
only_min_trials = False
fig = pf.custom_graph_template(x_title='', y_title='', height=800, width=1200,
                               rows=3, columns=6, shared_x=True, shared_y=True,
                               titles=[f'Day {x}' for x in np.arange(1, len(np.unique(middle_aged_df['day']))+1)])
for idx, day in enumerate(np.unique(middle_aged_df['day'])):
    plot_data = middle_aged_df[middle_aged_df['day'] == day]

    if only_min_trials:
        trial_lengths = []
        for group in plot_data.columns[:-1]:
            trial_lengths.append(len(plot_data.loc[:, group].values[0]))
        min_trials = np.min(trial_lengths)
    else:
        min_trials = -1
    
    for ind, group in enumerate(plot_data.columns[:-1]):
        group_data = plot_data.loc[:, group].values[0][:min_trials]
        upper = plot_data.loc[:, group].values[0][:min_trials] + plot_data.loc[:, group].values[1][:min_trials]
        lower = plot_data.loc[:, group].values[0][:min_trials] - plot_data.loc[:, group].values[1][:min_trials]
        x_data = np.arange(1, len(group_data)+1) * bin_size
        if idx < 6:
            row, col = 1, idx + 1
        elif (idx >= 6) & (idx < 12):
            row, col = 2, idx - 5
        else:
            row, col = 3, idx - 11
        fig.add_trace(go.Scatter(x=x_data, y=group_data, mode='lines', opacity=opacity,
                                line_color=group_colors_dict[group], showlegend=False,
                                legendgroup=group, name=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Upper Bound', x=x_data, y=upper, mode='lines',
                                 marker=dict(color=error_colors[ind]), line=dict(width=0), showlegend=False, 
                                 legendgroup=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Lower Bound', x=x_data, y=lower, marker=dict(color=error_colors[ind]),
                                 line=dict(width=0), mode='lines', fillcolor=error_colors[ind], fill='tonexty', 
                                 showlegend=False, legendgroup=group), row=row, col=col)
fig.add_hline(y=75, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy', range=[0, 100], dtick=25, col=1)
fig.update_xaxes(title='Trial', row=3)
fig.show()
if only_min_trials:
    fig.write_image(pjoin(fig_path, 'lick_acc_min_trials_middle_aged.png'))
else:
    fig.write_image(pjoin(fig_path, 'lick_acc_across_middle_aged.png'))

In [ ]:
group_of_interest = 'Aged'
group_data = {'Aged Thresh1': [], 'Aged Thresh5': [], 'day': []}
for day in np.unique(trial_acc['day']):
    group_data['Aged Thresh1'].append(np.nanmean(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Aged Thresh5'].append(np.nanmean(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0))
    group_data['Aged Thresh1'].append(np.std(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_1'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['Aged Thresh5'].append(np.std(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series), axis=0, ddof=1).to_numpy() / np.sqrt(len(trial_acc['trial_acc_5'][(trial_acc['group'] == group_of_interest) & (trial_acc['day'] == day)].apply(pd.Series))))
    group_data['day'].append(day)
    group_data['day'].append(day)
aged_df = pd.DataFrame(group_data)

In [ ]:
## For lick accuracy fifth lick
only_min_trials = False
fig = pf.custom_graph_template(x_title='', y_title='', height=800, width=1200,
                               rows=3, columns=6, shared_x=True, shared_y=True,
                               titles=[f'Day {x}' for x in np.arange(1, len(np.unique(aged_df['day']))+1)])
for idx, day in enumerate(np.unique(aged_df['day'])):
    plot_data = aged_df[aged_df['day'] == day]

    if only_min_trials:
        trial_lengths = []
        for group in plot_data.columns[:-1]:
            trial_lengths.append(len(plot_data.loc[:, group].values[0]))
        min_trials = np.min(trial_lengths)
    else:
        min_trials = -1
    
    for ind, group in enumerate(plot_data.columns[:-1]):
        group_data = plot_data.loc[:, group].values[0][:min_trials]
        upper = plot_data.loc[:, group].values[0][:min_trials] + plot_data.loc[:, group].values[1][:min_trials]
        lower = plot_data.loc[:, group].values[0][:min_trials] - plot_data.loc[:, group].values[1][:min_trials]
        x_data = np.arange(1, len(group_data)+1) * bin_size
        if idx < 6:
            row, col = 1, idx + 1
        elif (idx >= 6) & (idx < 12):
            row, col = 2, idx - 5
        else:
            row, col = 3, idx - 11
        fig.add_trace(go.Scatter(x=x_data, y=group_data, mode='lines', opacity=opacity,
                                line_color=group_colors_dict[group], showlegend=False,
                                legendgroup=group, name=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Upper Bound', x=x_data, y=upper, mode='lines',
                                 marker=dict(color=error_colors[ind]), line=dict(width=0), showlegend=False, 
                                 legendgroup=group), row=row, col=col)
        fig.add_trace(go.Scatter(name='Lower Bound', x=x_data, y=lower, marker=dict(color=error_colors[ind]),
                                 line=dict(width=0), mode='lines', fillcolor=error_colors[ind], fill='tonexty', 
                                 showlegend=False, legendgroup=group), row=row, col=col)
fig.add_hline(y=75, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy', range=[0, 100], dtick=25, col=1)
fig.update_xaxes(title='Trial', row=3)
fig.show()
if only_min_trials:
    fig.write_image(pjoin(fig_path, 'lick_acc_min_trials_aged.png'))
else:
    fig.write_image(pjoin(fig_path, 'lick_acc_across_aged.png'))

### Look at probe accuracy.

In [ ]:
## Looking at first licks
circletrack_results = {'mouse': [], 'day': [], 'group': [], 'session': [], 'lick_accuracy_thresh1': [], 'lick_accuracy_thresh5': []}
for mouse in os.listdir(circle_path)[:-1]: ## somehow a .DS_Store appeared????????
    mouse_path = pjoin(circle_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    for idx, session in enumerate(os.listdir(mouse_path)):
            behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
            if any(behav['probe']):
                behav = behav[behav['probe']]
                reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
                pc_thresh1 = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=1, by_trials=False)
                pc_thresh5 = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=5, by_trials=False)
                circletrack_results['mouse'].append(mouse)
                circletrack_results['day'].append(idx+1)
                circletrack_results['group'].append(group)
                circletrack_results['session'].append(np.unique(behav['session'])[0])
                circletrack_results['lick_accuracy_thresh1'].append(pc_thresh1)
                circletrack_results['lick_accuracy_thresh5'].append(pc_thresh5)
ct_df = pd.DataFrame(circletrack_results)

In [ ]:
y_variable = 'lick_accuracy_thresh1'
sess_pairs = [('A', 'AP'), ('B', 'BP'), ('C', 'CP'), ('D', 'DP')]
fig = pf.custom_graph_template(x_title='', y_title='', rows=1, columns=4, titles=['A', 'B', 'C', 'D'], 
                               width=1000, shared_y=True)
for idx, pair in enumerate(sess_pairs):
    loop_data = ct_df[(ct_df['session'] == pair[0]) | (ct_df['session'] == pair[1])]
    loop_data = loop_data.replace({pair[0]: 'First', pair[1]: 'Last'})
    avg_data = loop_data.groupby(['session', 'group'], as_index=False).agg({y_variable: ['mean', 'sem']})
    for group in np.unique(avg_data['group']):
        gdata = avg_data[avg_data['group'] == group]
        fig.add_trace(go.Scatter(x=gdata['session'], y=gdata[y_variable]['mean'], name=group,
                                 marker_color=group_colors_dict[group], showlegend=False, legendgroup=group,
                                 error_y=dict(type='data', array=gdata[y_variable]['sem'])), row=1, col=idx+1)
fig['data'][0]['showlegend'] = True
fig['data'][1]['showlegend'] = True
fig['data'][2]['showlegend'] = True
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy (%)', col=1)
fig.show()
fig.write_image(pjoin(fig_path, 'thresh1_probe_acc.png'))

In [ ]:
y_variable = 'lick_accuracy_thresh5'
sess_pairs = [('A', 'AP'), ('B', 'BP'), ('C', 'CP'), ('D', 'DP')]
fig = pf.custom_graph_template(x_title='', y_title='', rows=1, columns=4, titles=['A', 'B', 'C', 'D'], 
                               width=1000, shared_y=True)
for idx, pair in enumerate(sess_pairs):
    loop_data = ct_df[(ct_df['session'] == pair[0]) | (ct_df['session'] == pair[1])]
    loop_data = loop_data.replace({pair[0]: 'First', pair[1]: 'Last'})
    avg_data = loop_data.groupby(['session', 'group'], as_index=False).agg({y_variable: ['mean', 'sem']})
    for group in np.unique(avg_data['group']):
        gdata = avg_data[avg_data['group'] == group]
        fig.add_trace(go.Scatter(x=gdata['session'], y=gdata[y_variable]['mean'], name=group,
                                 marker_color=group_colors_dict[group], showlegend=False, legendgroup=group,
                                 error_y=dict(type='data', array=gdata[y_variable]['sem'])), row=1, col=idx+1)
fig['data'][0]['showlegend'] = True
fig['data'][1]['showlegend'] = True
fig['data'][2]['showlegend'] = True
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy (%)', col=1)
fig.show()
fig.write_image(pjoin(fig_path, 'thresh5_probe_acc.png'))

### Days to Criteria

In [ ]:
todays_mazes = pd.read_csv(f'../../../{project_dir}/{experiment_dir}/maze_yml/{experiment_dir} - TodaysMazes.csv')
melt_df = todays_mazes.melt(id_vars='Mouse')
finished_mice = np.unique(melt_df['Mouse'][melt_df['value'] == 'DONE'])
mdata = pd.DataFrame()
for mouse in finished_mice:
    loop_data = todays_mazes[todays_mazes['Mouse'] == mouse]
    mdata = pd.concat([mdata, loop_data])
melt_df = mdata.melt(id_vars='Mouse')
melt_df = melt_df[~pd.isna(melt_df['value'])]
melt_df = melt_df[(melt_df['value'] != 'AP') & (melt_df['value'] != 'BP') & (melt_df['value'] != 'CP') & (melt_df['value'] != 'DP') & (melt_df['value'] != 'DONE')]

In [ ]:
days_to = melt_df.groupby(['Mouse', 'value'], as_index=False).agg({'value': 'count'})
mouse_days_to = pd.DataFrame()
for mouse in np.unique(days_to['Mouse']):
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    loop_data = days_to[days_to['Mouse'] == mouse]
    loop_data.loc[:, 'context'] = ['A', 'B', 'C', 'D']
    loop_data.loc[:, 'group'] = group
    mouse_days_to = pd.concat([mouse_days_to, loop_data])
nonA_data = mouse_days_to[mouse_days_to['context'] != 'A']

In [ ]:
nonA_data = mouse_days_to[mouse_days_to['context'] != 'A']
fig = pf.custom_graph_template(x_title='', y_title='Days to Criteria')
for group in np.unique(nonA_data['group']):
    gdata = nonA_data[nonA_data['group'] == group]
    avg_data = gdata.groupby(['context'], as_index=False).agg({'value': ['mean', 'sem']})
    fig.add_trace(go.Scatter(x=avg_data['context'], y=avg_data['value']['mean'], name=group, marker_color=group_colors_dict[group],
                             error_y=dict(type='data', array=avg_data['value']['sem'])))
# fig.update_yaxes(range=[0, 6.5])
fig.show()
fig.write_image(pjoin(fig_path, 'days_to_criteria.png'))

### Create a velocity distribution for all three groups.

In [88]:
young_v = []
middle_aged_v = []
aged_v = []
downsample_factor = 42
for mouse in os.listdir(circle_path)[:-1]:
    mouse_path = pjoin(circle_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        x_cm, y_cm = ctb.convert_to_cm(x=behav['x'], y=behav['y'], update_dataframe=False)
        velocity, running = pc.define_running_epochs(x_cm[::downsample_factor], y_cm[::downsample_factor], behav['t'][::downsample_factor])
        avg_vel = np.mean(velocity)
        if group == 'Young Adult':
            young_v.append(avg_vel)
        elif group == 'Middle Aged':
            middle_aged_v.append(avg_vel)
        else:
            aged_v.append(avg_vel)

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Behavior/MultiCon_Aging2/../../place_cells.py:44: RuntimeWarning:

divide by zero encountered in divide

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Behavior/MultiCon_Aging2/../../place_cells.py:44: RuntimeWarning:

invalid value encountered in divide



In [89]:
fig = pf.custom_graph_template(x_title='Average Velocity (cm/s)', y_title='Probability', height=500, width=500)
fig.add_trace(go.Histogram(x=young_v, marker_color=group_colors_dict['Young Adult'], 
                           name='Young Adult', opacity=0.6, histnorm='probability'))
fig.add_trace(go.Histogram(x=middle_aged_v, marker_color=group_colors_dict['Middle Aged'], 
                           name='Middle Aged', opacity=0.6, histnorm='probability'))
fig.add_trace(go.Histogram(x=aged_v, marker_color=group_colors_dict['Aged'], 
                           name='Aged', opacity=0.6, histnorm='probability'))
fig.update_layout(barmode='overlay')
fig.show()
fig.write_image(pjoin(fig_path, 'velocity_distribution.png'))

In [90]:
young_v = np.asarray(young_v)[~np.isnan(young_v)]
anova_df = pd.DataFrame({'young': young_v,
                         'middle_aged': middle_aged_v[0:len(young_v)],
                         'aged': aged_v[0:len(young_v)]})
anova_melt = anova_df.melt(value_vars=['young', 'middle_aged', 'aged'], value_name='velocity', var_name='group')
print(pg.homoscedasticity(anova_melt, dv='velocity', group='group'))
anova_melt.welch_anova(dv='velocity', between='group')

                W          pval  equal_var
levene  25.672622  2.014414e-11      False


,Source,ddof1,ddof2,F,p-unc,np2
0,group,2,388.727056,56.965576,2.011806e-22,0.167775


In [91]:
pg.pairwise_gameshowell(anova_melt, dv='velocity', between='group')

,A,B,mean(A),mean(B),diff,se,T,df,pval,hedges
0,aged,middle_aged,10.311236,12.130511,-1.819274,0.355803,-5.113145,355.111192,1.552046e-06,-0.510350
1,aged,young,10.311236,13.947997,-3.636761,0.351480,-10.346981,347.577235,5.306866e-14,-1.032747
2,middle_aged,young,12.130511,13.947997,-1.817486,0.282033,-6.444226,397.412779,1.009694e-09,-0.643208


### Separate velocity distributions by days.

In [101]:
vel_dict = {'mouse': [], 'group': [], 'day': [], 'context_day': [], 'session': [], 'avg_velocity': [], 'max_velocity': [], 'vel_values': []}
downsample_factor = 42
for mouse in os.listdir(circle_path)[:-1]:
    mouse_path = pjoin(circle_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    session_id = np.nan
    day_counter = 1
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        x_cm, y_cm = ctb.convert_to_cm(x=behav['x'], y=behav['y'], update_dataframe=False)
        velocity, running = pc.define_running_epochs(x_cm[::downsample_factor], y_cm[::downsample_factor], behav['t'][::downsample_factor])
        vel_dict['mouse'].append(mouse)
        vel_dict['group'].append(group)
        vel_dict['day'].append(idx+1)

        if np.unique(behav['session'])[0] != session_id:
            day_counter = 1
            session_id = np.unique(behav['session'])[0]
            vel_dict['context_day'].append(day_counter)
        else:
            day_counter += 1
            session_id = np.unique(behav['session'])[0]
            vel_dict['context_day'].append(day_counter)

        vel_dict['session'].append(np.unique(behav['session'])[0])
        vel_dict['avg_velocity'].append(np.mean(velocity))
        vel_dict['max_velocity'].append(np.max(velocity))
        vel_dict['vel_values'].append(velocity)
vel = pd.DataFrame(vel_dict)

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Behavior/MultiCon_Aging2/../../place_cells.py:44: RuntimeWarning:

divide by zero encountered in divide

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/MultiCon_Behavior/MultiCon_Aging2/../../place_cells.py:44: RuntimeWarning:

invalid value encountered in divide



In [124]:
## Plot average velocity for each group across days in each context
session_list = ['A', 'B', 'C', 'D']
vel_contexts = pd.DataFrame()
for session in session_list:
    loop_data = vel[vel['session'] == session]
    vel_contexts = pd.concat([vel_contexts, loop_data], ignore_index=True)
fig = pf.custom_graph_template(x_title='', y_title='', rows=1, columns=4, width=1000, 
                               shared_y=True, shared_x=True, titles=session_list)
for group in vel_contexts['group'].unique():
    gdata = vel_contexts[vel_contexts['group'] == group].groupby(['context_day', 'session'], as_index=False).agg({'avg_velocity': ['mean', 'sem']})
    for idx, sess in enumerate(gdata['session'].unique()):
        sess_data = gdata[gdata['session'] == sess]
        fig.add_trace(go.Scatter(x=sess_data['context_day'], y=sess_data['avg_velocity']['mean'], name=group,
                                 line_color=group_colors_dict[group], legendgroup=group, showlegend=False,
                                 error_y=dict(type='data', array=sess_data['avg_velocity']['sem'])), row=1, col=idx+1)
fig['data'][0]['showlegend'] = True
fig['data'][5]['showlegend'] = True
fig['data'][11]['showlegend'] = True
fig.update_xaxes(title='Day in Context')
fig.update_yaxes(title='Average Velocity (cm/s)')
fig.show()

### Lick accuracy heatmap.

In [ ]:
## Looking at first licks
lick_thresholds = [1, 2, 3, 4, 5]
circletrack_results = {'mouse': [], 'day': [], 'context_day': [], 'group': [], 'session': [], 'lick_accuracy_thresh1': [], 'lick_accuracy_thresh2': [],
                       'lick_accuracy_thresh3': [], 'lick_accuracy_thresh4': [], 'lick_accuracy_thresh5': []}
for mouse in os.listdir(circle_path)[:-1]: ##.DS_store
    mouse_path = pjoin(circle_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    session_id = np.nan
    day_counter = 1
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        behav = behav[~behav['probe']]
        reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
        circletrack_results['mouse'].append(mouse)
        circletrack_results['day'].append(idx+1)
        circletrack_results['group'].append(group)
        circletrack_results['session'].append(np.unique(behav['session'])[0])
        for thresh in lick_thresholds:
            lick_accuracy = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=thresh, by_trials=False)
            circletrack_results[f'lick_accuracy_thresh{thresh}'].append(lick_accuracy)

        if np.unique(behav['session'])[0] != session_id:
            day_counter = 1
            session_id = np.unique(behav['session'])[0]
            circletrack_results['context_day'].append(day_counter)
        else:
            day_counter += 1
            session_id = np.unique(behav['session'])[0]
            circletrack_results['context_day'].append(day_counter)
ct_df = pd.DataFrame(circletrack_results)
## Average for each lick accuracy threshold, excluding final probe days
select_sessions = ct_df[(ct_df['session'] != 'AP') & (ct_df['session'] != 'BP') & (ct_df['session'] != 'CP') & (ct_df['session'] != 'DP')]
avg_data = select_sessions.groupby(['context_day', 'group', 'session'], as_index=False).agg('mean')

In [ ]:
## Young adult mice
young_acc = avg_data[avg_data['group'] == 'Young Adult'].reset_index(drop=True)
fig = pf.custom_graph_template(x_title='', y_title='', rows=1, columns=4, 
                               width=1200, titles=['A', 'B', 'C', 'D'])

for sess_idx, session in enumerate(np.unique(young_acc['session'])):
    sess_data = young_acc[young_acc['session'] == session].reset_index(drop=True)
    arr = np.empty((len(lick_thresholds), sess_data.shape[0]))
    arr.fill(np.nan)
    for idx, col in enumerate(sess_data.columns[4:]):
        arr[idx] = sess_data[col].to_numpy()
    x_axis = np.arange(1, arr.shape[1]+1)
    fig.add_trace(go.Heatmap(x=x_axis, y=lick_thresholds, z=arr, 
                             coloraxis='coloraxis'), row=1, col=sess_idx+1)
fig.update_xaxes(title='Day in Context', row=1)
fig.update_yaxes(title='Lick Threshold', col=1)
fig.update_layout(coloraxis={'colorscale': 'viridis'})
fig.show()

In [ ]:
## Middle aged mice
acc = avg_data[avg_data['group'] == 'Middle Aged'].reset_index(drop=True)
fig = pf.custom_graph_template(x_title='', y_title='', rows=1, columns=4, 
                               width=1200, titles=['A', 'B', 'C', 'D'])

for sess_idx, session in enumerate(np.unique(acc['session'])):
    sess_data = acc[acc['session'] == session].reset_index(drop=True)
    arr = np.empty((len(lick_thresholds), sess_data.shape[0]))
    arr.fill(np.nan)
    for idx, col in enumerate(sess_data.columns[4:]):
        arr[idx] = sess_data[col].to_numpy()
    x_axis = np.arange(1, arr.shape[1]+1)
    fig.add_trace(go.Heatmap(x=x_axis, y=lick_thresholds, z=arr, 
                             coloraxis='coloraxis'), row=1, col=sess_idx+1)
fig.update_xaxes(title='Day in Context', row=1)
fig.update_yaxes(title='Lick Threshold', col=1)
fig.update_layout(coloraxis={'colorscale': 'viridis'})
fig.show()

In [ ]:
acc = avg_data[avg_data['group'] == 'Aged'].reset_index(drop=True)
fig = pf.custom_graph_template(x_title='', y_title='', rows=1, columns=4, 
                               width=1200, titles=['A', 'B', 'C', 'D'])

for sess_idx, session in enumerate(np.unique(acc['session'])):
    sess_data = acc[acc['session'] == session].reset_index(drop=True)
    arr = np.empty((len(lick_thresholds), sess_data.shape[0]))
    arr.fill(np.nan)
    for idx, col in enumerate(sess_data.columns[4:]):
        arr[idx] = sess_data[col].to_numpy()
    x_axis = np.arange(1, arr.shape[1]+1)
    fig.add_trace(go.Heatmap(x=x_axis, y=lick_thresholds, z=arr, 
                             coloraxis='coloraxis'), row=1, col=sess_idx+1)
fig.update_xaxes(title='Day in Context', row=1)
fig.update_yaxes(title='Lick Threshold', col=1)
fig.update_layout(coloraxis={'colorscale': 'viridis'})
fig.show()

In [ ]:
fig = pf.custom_graph_template(x_title='', y_title='', rows=3, columns=4, shared_y=True, shared_x=True,
                               height=1000, width=1200, titles=['A', 'B', 'C', 'D'])
for g_idx, group in enumerate(['Young Adult', 'Middle Aged', 'Aged']):
    acc = avg_data[avg_data['group'] == group].reset_index(drop=True)
    for sess_idx, session in enumerate(np.unique(acc['session'])):
        sess_data = acc[acc['session'] == session].reset_index(drop=True)
        arr = np.empty((len(lick_thresholds), sess_data.shape[0]))
        arr.fill(np.nan)
        for idx, col in enumerate(sess_data.columns[4:]):
            arr[idx] = sess_data[col].to_numpy()
        x_axis = np.arange(1, arr.shape[1]+1)
        fig.add_trace(go.Heatmap(x=x_axis, y=lick_thresholds, z=arr, 
                                coloraxis='coloraxis'), row=g_idx+1, col=sess_idx+1)
fig.update_xaxes(title='Day in Context', row=3)
fig.update_yaxes(title='Lick Threshold', col=1)
fig.update_layout(coloraxis={'colorscale': 'viridis'})
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy_heatmaps.png'))

### Reminder day for IHC

In [125]:
circletrack_results = {'mouse': [], 'day': [], 'group': [], 'session': [], 'lick_accuracy_thresh1': [], 
                       'lick_accuracy_thresh5': [], 'rewards': []}
for mouse in os.listdir(circle_path)[:-1]: ## somehow a .DS_Store appeared????????
    mouse_path = pjoin(circle_path, mouse)
    group = 'Young Adult' if mouse in young_adult else 'Middle Aged' if mouse in middle_aged else 'Aged'
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        if (np.unique(behav['session'])[0] != 'DT' ) & (np.unique(behav['session'])[0] != 'CT' ):
            pass
        else:
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            pc_thresh1 = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=1, by_trials=False)
            pc_thresh5 = ctb.lick_accuracy(behav, port_one=reward_one, port_two=reward_two, lick_threshold=5, by_trials=False)
            circletrack_results['mouse'].append(mouse)
            circletrack_results['day'].append(idx+1)
            circletrack_results['group'].append(group)
            circletrack_results['session'].append(np.unique(behav['session'])[0])
            circletrack_results['lick_accuracy_thresh1'].append(pc_thresh1)
            circletrack_results['lick_accuracy_thresh5'].append(pc_thresh5)
            circletrack_results['rewards'].append(np.sum(behav['water']))
ct_df = pd.DataFrame(circletrack_results)

In [126]:
## Plot lick accuracy thresh5 for all groups
var = 'lick_accuracy_thresh5'
avg_data = ct_df.groupby(['group'], as_index=False).agg({var: ['mean', 'sem']})
fig = pf.custom_graph_template(x_title='', y_title='Lick Accuracy (%)')
for group in np.unique(avg_data['group']):
    gdata = avg_data[avg_data['group'] == group]
    fig.add_trace(go.Scatter(x=gdata['group'], y=gdata[var]['mean'], mode='markers', name=group, showlegend=False,
                            marker_color=group_colors_dict[group], error_y=dict(type='data', array=gdata[var]['sem']),
                            marker_size=5))
for mouse in np.unique(ct_df['mouse']):
    mdata = ct_df[ct_df['mouse'] == mouse]
    group = mdata['group'].values[0]
    fig.add_trace(go.Scatter(x=mdata['group'], y=mdata[var], mode='markers', name=group, showlegend=False,
                             marker_color=group_colors_dict[group]))

fig.show()

In [127]:
## Plot rewards for all groups
var = 'rewards'
avg_data = ct_df.groupby(['group'], as_index=False).agg({var: ['mean', 'sem']})
fig = pf.custom_graph_template(x_title='', y_title='Rewards')
for group in np.unique(avg_data['group']):
    gdata = avg_data[avg_data['group'] == group]
    fig.add_trace(go.Scatter(x=gdata['group'], y=gdata[var]['mean'], mode='markers', name=group, showlegend=False,
                            marker_color=group_colors_dict[group], error_y=dict(type='data', array=gdata[var]['sem'])))
for mouse in np.unique(ct_df['mouse']):
    mdata = ct_df[ct_df['mouse'] == mouse]
    group = mdata['group'].values[0]
    fig.add_trace(go.Scatter(x=mdata['group'], y=mdata[var], mode='markers', name=group, showlegend=False,
                             marker_color=group_colors_dict[group]))

fig.show()